# mDC proportion analysis: Differential gene expression by NEBULA 
Use notebook in conjunction with R files provided for each step of the analysis
Initial steps to generate Figure 6 B-C, Extended Figure A-C (in another notebook)

- Author: CW
- File input: 1_RNA_all.rds
- Output files: mDC_proportion_analysis/1_NEBULA_celltype_objs, 1_NEBULA_subtype_objs, 2_NEBULA2_celltype_res, 2_NEBULA_subtype_res, 3_NEBULA2_celltype_summaries, 3_NEBULA_subtype_summaries
- Last updated: 04/13/24

In [7]:
library('dplyr')
library('tidyr')
library('ggplot2')
library('Matrix')
library(tibble)
library(ggpubr)

Warning message:
“package ‘dplyr’ was built under R version 4.3.2”

Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack




In [ ]:
library(nebula)

# 1. create NEBULA objects
see "1_create_NEBULA_objects.R" and "1_create_NEBULA_objects.sh" to run in parallel

#### FUNCTION

In [1]:
create_NEBULA_object <- function(obj, cell_type, path_to_save) {
    
    #if path to save is not a path yet, make a directory
    if (!dir.exists(path_to_save)){
        dir.create(path_to_save)
    } 
    else {
        print("Dir already exists!")
    }
    
    #obj = seurat object with correct metadata

    if(cell_type == 'celltype'){
        ct_list <- unique(obj@meta.data$cell_type)
    }
    
    if(cell_type == 'subtype'){
        ct_list <- unique(obj@meta.data$cell_subtype)
        
    }

    for (ct in ct_list) {
    	print(ct)
        nebula_df <- vector(mode = "list")

        #subset cluster by cell type 

        if(cell_type == 'celltype') {
            cluster <- subset(obj, subset = (cell_type == ct))
        }

        if(cell_type == 'subtype') {
          {
            	cluster <- subset(obj, subset = (cell_subtype == ct))
        	}
        }
    
        #head(cluster@meta.data)

        #calculate offset - using nFeature_RNA
        nebula_df$offset <-  cluster@meta.data$nCount_RNA

        nebula_df$patient_sample <- cluster@meta.data$patient_sample

        #extract raw count matrix 
        DefaultAssay(cluster) <- 'RNA'
        count_mtx <- GetAssayData(object = cluster, slot = "counts")

        nebula_df$count <- count_mtx

        #design matrix for cluster 
        nebula_df$pred <- cluster@meta.data[c("age", "sex", "sample_ID_long",
                    "tissue","treatment_group","state", "subtype", "percent_mt",
                    "percent_ribo", "BRAF", "NRAS", "is_mDC_high")]

        #df = model.matrix(~mDC_low_vs_high, data = obj$pred)

        #save object 

        #if any backslashes in the cell type name, replace with underscore
        new_ct <- gsub("/", "_", ct)
        new_ct <- gsub(" ", "_", new_ct)
        new_ct <- gsub("-", "_", new_ct)
        full_path <- paste0(path_to_save, new_ct, '.rds')

        #print(full_path)
        saveRDS(nebula_df, full_path)
    
    }

}

#### create NEBULA objects by celltypes and subtypes with all samples 

In [ ]:
obj <- readRDS('/path_to_file/1_RNA_all.rds')

#create NEBULA object to run later 
#1. ALL samples 
#reannotated
create_NEBULA_object(obj, 'celltype', path_to_save = './mDC_proportion_analysis/1_NEBULA_celltype_objs/')

#subtyped on all samples 
create_NEBULA_object(obj, 'subtyped', './mDC_proportion_analysis/1_NEBULA_subtype_objs/')

## 2. Run NEBULA
see "2_run_NEBULA.R" and "2_run_NEBULA2_celltypes.sh, 2_run_NEBULA_subtypes.sh" to run in parallel

In [8]:
library(stringr)
library(nebula3)

- in_file = one object from 1_NEBULA_celltype_objs or 1_NEBULA_subtype_objs
- out_file = path to where immediate NEBULA results are stored 

- part = 'two' for celltypes: new version of NEBULA (NEBULA2) 
- part = 'one' for subtpyes: first version of NEBULA (NEBULA1)

In [ ]:
#time 
start_time <- Sys.time()

df <- readRDS(in_file)

#####PRE-PROCESSING STEPS: Filter, select covariates, make design matrix####### 

#1. change response of patient 479_1
if('479_1' %in% unique(df$patient_sample)) {
  df$pred[df$patient_sample == '479_1',]$response_short = 'R'
}

#2. subset out patient 1122_1
index = which(df$patient_sample !='1122_1')

#Processing steps: 
#make all variables excluding that patient
count = df$count[,index] #number of counts by cell 
id = df$patient_sample[index]
offset = df$offset[index]

#select predictor variable for mDC proportion analysis 
pred = df$pred[index,c('is_mDC_high','age','sex', 'percent_mt','percent_ribo',
                        'tissue', 'state', 'treatment_group')]

#format the covariates before encoding into the design matrix 
#for with ICI samples - with ICI, ICI only, antiPD1
pred$treatment_group_new <- "other"
if(dim(pred[pred$treatment_group %in% c('ICI_PD1','ICI_combo', 'targeted_plus_ICI', 
                                        'other_plus_ICI'),])[1] > 0){
  pred[pred$treatment_group %in% c('ICI_PD1','ICI_combo', 'targeted_plus_ICI', 
                                   'other_plus_ICI'),]$treatment_group_new <-  "with_ICI"
}
if(dim(pred[pred$treatment_group %in% c('ICI_PD1','ICI_combo'),])[1] > 0){
  pred[pred$treatment_group %in% c('ICI_PD1','ICI_combo'),]$treatment_group_new <-"ICI_only"
}
if('ICI_PD1' %in% pred$treatment_group) {
  pred[pred$treatment_group == 'ICI_PD1',]$treatment_group_new <- "ICI_PD1"
}
pred$treatment_group <- pred$treatment_group_new
pred$treatment_group_new <- NULL

#all celltypes and groups, make tissue three categories 
if('skin' %in% pred$tissue) {
  pred[pred$tissue == 'skin',]$tissue <- 'skin'
}
if('lymph' %in% pred$tissue) {
  pred[pred$tissue == 'lymph', ]$tissue <- 'lymph'
}
if(length(pred[!pred$tissue %in% c('skin','lymph'),]$tissue > 0)){
  pred[!pred$tissue %in% c('skin', 'lymph') , ]$tissue <- 'other' 
}

### check for apparent collinearity: check for every column that there are two or more levels 
for(col in colnames(pred)) {
  if(dim(unique(pred[col]))[1] == 1) {
    pred[col] <- NULL
  }
}
#remove columns that lead to collinearity (correlation of 1 with another column) - skip 

#Prediction columns 
print(paste0('prediction columns ', colnames(pred)))

#####MAKE DESIGN MATRIX#####
pred_design_matrix = model.matrix(~.,pred)

#####RUN NEBULA1 for subtypes and save results#####
if(part == 'one') {
  re = nebula(count,id,offset=offset,pred=pred_design_matrix,ncore=4, 
    separation = 'normal')

  #save results from nebula part 1
  sep_in_file = strsplit(in_file, split = "/")
  celltype = tail(sep_in_file[[1]], 1)

  #save this result with the regularization
  if(!dir.exists(paste0('./mDC_proportion_analysis/2_NEBULA_subtype_res/'))) {
    dir.create(paste0('./mDC_proportion_analysis/2_NEBULA_subtype_res/'), recursive = TRUE) 
  }

  saveRDS(re, paste0('./mDC_proportion_analysis/2_NEBULA_subtype_res/', celltype))
    
  end_time <- Sys.time()
  print(paste0("Time to run nebula1: ", end_time - start_time))

} else { 
##### RUN NEBULA2 - no regularization on nebula1 - on celltypes and save results #####
  re = nebula(count,id,offset=offset,pred=pred_design_matrix,ncore=4)
  #topdeg = 1:nrow(re$summary)
  #Changes with NEBULA 1.0.9
  topdeg = which(re$overdispersion[,2]>0.001)

  #scale number of cores to use with the number of cells 
  num_cores = 2
  if(length(index) > 40000) {
    num_cores = 6
  }

  re2 = nebula_mi(nebula=re, index=2, nmc=300, gimp=topdeg,count=count,id=id,offset=offset,pred=pred_design_matrix,ncore=num_cores)

  #save results from nebula part 2
  #did this separately 
  if(!dir.exists(paste0('./mDC_proportion_analysis/2_NEBULA2_celltype_res/'))) {
    dir.create(paste0('./mDC_proportion_analysis/2_NEBULA2_celltype_res/'), recursive = TRUE)
  }
    
  sep_in_file = strsplit(in_file, split = "/")
  celltype = tail(sep_in_file[[1]], 1)
    
  saveRDS(re2,paste0('./mDC_proportion_analysis/2_NEBULA2_celltype_res/', celltype))
  end_time <- Sys.time()

  print(paste0("Time to run nebula2: ", end_time - start_time))

  end_time <- Sys.time()

In [3]:
### used job scripts to run via SLURM, 2_run_NEBULA2_celltypes.sh and 2_run_NEBULA_subtypes.sh

## 3. Summarize NEBULA results  - done here


In [1]:
library(dplyr)

Warning message:
“package ‘dplyr’ was built under R version 4.3.2”

Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




### A. For NEBULA1 results - subtypes

In [4]:
summarize_NEBULA_res_1 <- function(path_to_NEBULA_analysis, folder ) { 
    #path_to_NEBULA_res <-'/home/cbw3/data/differential_gene_expression/mDC_analysis_109/' #NEBULA_reannotated_w_ICI_res/part1/'
    #print(path_to_NEBULA_res)
    
    for(f in list.files(paste0(path_to_NEBULA_analysis, folder))) {
        celltype <- strsplit(f, '[.]')[[1]][1]
        if(!file.exists(paste0(path_to_NEBULA_analysis, folder, '/', celltype, '.csv'))) {
            #print(file)
            res <- readRDS(paste0(path_to_NEBULA_analysis, folder, '/', celltype, '.csv'))
        
            if(dim(res$summary)[1] > 0 & 'p_is_mDC_high1' %in% colnames(res$summary)) {

                #make a new column with adjusted p values (with FDR)
                res$summary$pval_adj <- p.adjust(res$summary$p_is_mDC_high1, 'fdr')
                volcano_df <- res$summary %>% arrange(desc(logFC_is_mDC_high1))

                #change the column names to generic names for easier plotting 
                colnames(volcano_df)[colnames(volcano_df) == 'gene'] <- 'gene_symbol'
                colnames(volcano_df)[colnames(volcano_df) == 'logFC_is_mDC_high1'] <- 'logfc'
                colnames(volcano_df)[colnames(volcano_df) == 'p_is_mDC_high1'] <- 'pvalue'

                #print(colnames(volcano_df))

                volcano_df$diffexpressed = 'NO'

                #make arbitrary cutoffs for now 
                lf_cutoff = 0
                pval_cutoff = 0.05

                volcano_df$diffexpressed[volcano_df$logfc > lf_cutoff & volcano_df$pval_adj < pval_cutoff] <- 'UP'
                volcano_df$diffexpressed[volcano_df$logfc < -lf_cutoff & volcano_df$pval_adj < pval_cutoff] <- 'DOWN'

                volcano_df$pval_adj <- volcano_df$pval_adj + 1e-319
                #save volcano dfs as well

                #overwrite results with the new columns added 
                if(!dir.exists(paste0(path_to_NEBULA_analysis, '3_NEBULA_subtype_summaries'))) {
                    dir.create(paste0(path_to_NEBULA_analysis, '3_NEBULA_subtype_summaries'), recursive = TRUE)
                }

                write.csv(volcano_df, paste0(path_to_NEBULA_analysis, '3_NEBULA_subtype_summaries/', celltype, '.csv'))
                }
            }
        
        }
        
    }

- output: 3_NEBULA_subtype_summaries/

In [ ]:
path_to_NEBULA_analysis <- '/path_to_folder/mDC_proportion_analysis/'

#change folder 
folder <- '2_NEBULA_subtype_res'
summarize_NEBULA_res_1(path_to_NEBULA_analysis, folder)

### B. For NEBULA2 results - celltypes

In [5]:
summarize_NEBULA_res_2 <- function(path_to_NEBULA_analysis, folder ) { 
    
    for(f in list.files(paste0(path_to_NEBULA_analysis, folder))) {
        celltype <- strsplit(f, '[.]')[[1]][1]
        if(!file.exists(paste0(path_to_NEBULA_analysis, folder, '/', celltype, '.csv'))) {
            print(f)
            res <- readRDS(paste0(path_to_NEBULA_analysis, folder, '/', celltype, '.csv'))
            #print(f)
            if(dim(res$summary)[1] > 0) {

                #make a new column with adjusted p values (with FDR)
                res$summary$pval_adj <- p.adjust(res$summary$pvalue, 'fdr')
                volcano_df <- res$summary %>% arrange(desc(logfc))

                #change the column names to generic names for easier plotting 
                colnames(volcano_df)[colnames(volcano_df) == 'gene.gene'] <- 'gene_symbol'

                volcano_df$diffexpressed = 'NO'

                #make arbitrary cutoffs for now 
                lf_cutoff = 0
                pval_cutoff = 0.05

                volcano_df$diffexpressed[volcano_df$logfc > lf_cutoff & volcano_df$pval_adj < pval_cutoff] <- 'UP'
                volcano_df$diffexpressed[volcano_df$logfc < -lf_cutoff & volcano_df$pval_adj < pval_cutoff] <- 'DOWN'

                volcano_df$pval_adj <- volcano_df$pval_adj + 1e-319
                #save volcano dfs as well

                #overwrite results with the new columns added 
                if(!dir.exists(paste0(path_to_NEBULA_analysis, '3_NEBULA2_celltype_summaries'))) {
                    dir.create(paste0(path_to_NEBULA_analysis, '3_NEBULA2_celltype_summaries'), recursive = TRUE)
                }

                write.csv(volcano_df, paste0(path_to_NEBULA_analysis, '3_NEBULA2_celltype_summaries', celltype, '.csv'))
                } 
            }    
        }   
    }

- output: 3_NEBULA2_celltype_summaries/

In [7]:
path_to_NEBULA_analysis <- '/path_to_folder/mDC_proportion_analysis/'

#change folder 
folder <- '2_NEBULA_celltype_res'
summarize_NEBULA_res_2(path_to_NEBULA_analysis, folder)

[1] "Tumor.rds"

NEBULA subtypes: that do not work at all 
if part 1 does not work or has collinearity, then part 2 won't work 

all: 
 - mDC part 1: Tumor MYC, Tumor EMT III
 - mDC part 2: Memory_B.rds, Tumor___EMT_III.rds, Tumor___Hypoxia.rds, Tumor___MYC.rds, Tumor___NA.rds, Tumor___Secreted_I.rds, Tumor___Skin_pigmentation.rds, Tumor___Stress_(in_vitro).rds, Tumor___Translation_initiation.rds, Tumor___Unassigned.rds
 - response part 1: 'Memory_B.rds','Tumor___EMT_III.rds','Tumor___MYC.rds'
 - reponse part 2: 'Memory_B.rds', 'Tumor___EMT_III.rds', 'Tumor___Hypoxia.rds', 'Tumor___MYC.rds', 'Tumor___NA.rds''Tumor___Secreted_I.rds''Tumor___Skin_pigmentation.rds''Tumor___Stress_(in_vitro).rds''Tumor___Translation_initiation.rds'
 
## RERUN
- for ALL, part 2, try Tumor___NA again for mDC (8623967), Nebula 2

#mDC analysis: NEBULA subtype res - Part 1
- Tumor___EMT_III.rds all cells are from mDC low, "invalid 'type' (complex) of argument"
- Tumor___MYC.rds are cells from mDC low, "invalid 'type' (complex) of argument" 
- Tumor___Hypoxia.rds for NEBULA2, "invalid 'type' (complex) of argument" 
- Tumor___Stress_(in_vitro).rds has collinearity problem when all treatment groups
- Tumor__Secreted_I.rds all samples are targeted_plus_ICI  


In [9]:
sessionInfo()

R version 4.3.1 (2023-06-16)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS/LAPACK: /net/bmc-lab5/data/kellis/users/cbw3/conda/envs/new_r/lib/libopenblasp-r0.3.24.so;  LAPACK version 3.11.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: America/New_York
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] nebula3_1.0.9  stringr_1.5.0  ggpubr_0.6.0   tibble_3.2.1   Matrix_1.6-1.1
[6] ggplot2_3.4.4  tidyr_1.3.0    dplyr_1.1.4   

loaded via a namespace (and not attached):
 [1] gtable_0.3.4        rst